In [1]:
from datetime import datetime
import os
import sys

# import fire
# import gradio as gr
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

from utils.callbacks import Iteratorize, Stream
from utils.prompter import Prompter


device_no = 0

if torch.cuda.is_available():
    device = f"cuda:{device_no}"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:  # noqa: E722
    pass

# from google.colab import drive
# drive.mount('/content/drive')

# !mkdir adapter_model

# !cp -r /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b/adapter_model/ adapter_model

# template folder
# alpaca.json
# utils folder

# model_name = "amdnsr/llama-7b-hf"
base_model = "amdnsr/llama-7b-hf"






from utils.prompter import Prompter

add_eos_token: bool = False
cutoff_len = CUTOFF_LEN = 256
train_on_inputs = False
prompt_template_name: str = "alpaca"
prompter = Prompter(prompt_template_name)



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)



def evaluate(
    input_ids,
    # instruction,
    # input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    # max_new_tokens is set to 8, as for our model, only a single word prediction is needed
    max_new_tokens=32,
    stream_output=False,
    **kwargs,
):
    # prompt = prompter.generate_prompt(instruction, input)
    # inputs = tokenizer(prompt, return_tensors="pt")
    # input_ids = inputs["input_ids"].to(device)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )

    generate_params = {
        "input_ids": input_ids,
        "generation_config": generation_config,
        "return_dict_in_generate": True,
        "output_scores": True,
        "max_new_tokens": max_new_tokens,
    }

    if stream_output:
        # Stream the reply 1 token at a time.
        # This is based on the trick of using 'stopping_criteria' to create an iterator,
        # from https://github.com/oobabooga/text-generation-webui/blob/ad37f396fc8bcbab90e11ecf17c56c97bfbd4a9c/modules/text_generation.py#L216-L243.

        def generate_with_callback(callback=None, **kwargs):
            kwargs.setdefault(
                "stopping_criteria", transformers.StoppingCriteriaList()
            )
            kwargs["stopping_criteria"].append(
                Stream(callback_func=callback)
            )
            with torch.no_grad():
                model.generate(**kwargs)

        def generate_with_streaming(**kwargs):
            return Iteratorize(
                generate_with_callback, kwargs, callback=None
            )

        with generate_with_streaming(**generate_params) as generator:
            for output in generator:
                # new_tokens = len(output) - len(input_ids[0])
                decoded_output = tokenizer.decode(output)

                if output[-1] in [tokenizer.eos_token_id]:
                    break

                return prompter.get_response(decoded_output)
        return  # early return for stream_output

    # Without streaming
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return prompter.get_response(output)



def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    # result["labels"] = result["input_ids"].copy()

    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        # data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    # if not train_on_inputs:
    #     user_prompt = prompter.generate_prompt(
    #         data_point["instruction"], data_point["input"]
    #     )
    #     tokenized_user_prompt = tokenize(
    #         user_prompt, add_eos_token=add_eos_token
    #     )
    #     user_prompt_len = len(tokenized_user_prompt["input_ids"])

    #     if add_eos_token:
    #         user_prompt_len -= 1

    #     tokenized_full_prompt["labels"] = [
    #         -100
    #     ] * user_prompt_len + tokenized_full_prompt["labels"][
    #         user_prompt_len:
    #     ]  # could be sped up, probably
    return tokenized_full_prompt



print(datetime.now())


/home/kokil/anaconda3/envs/qlora_conda_3912_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kokil/anaconda3/envs/qlora_conda_3912_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda111_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 111
CUDA SETUP: Loading binary /home/kokil/anaconda3/envs/qlora_conda_3912_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda111_nocublaslt.so...


/home/kokil/anaconda3/envs/qlora_conda_3912_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/kokil/anaconda3/envs/qlora_conda_3912_env did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/kokil/anaconda3/envs/qlora_conda_3912_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/kokil/anaconda3/envs/qlora_conda_3912_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/kokil/anaconda3/envs/qlora_conda_3912_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarni

2024-02-12 20:33:39.066692


In [2]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    # quantization_config=bnb_config,
    # device_map={"": device_no},
    device_map="auto",
)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 33/33 [00:22<00:00,  1.45it/s]


In [3]:
model.config.use_cache = False

In [4]:
model.training

False

In [5]:
from transformers import LlamaTokenizer, LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained(base_model)
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

In [8]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [9]:
model.training

False

In [10]:
    # model = torch.compile(model)


In [11]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [13]:
model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16,
)
print("reached here")


In [14]:
dataset_name = "gab"
dataset_home = "/home/kokil/ahmad/benchmarking_llms/projects/qlora_training/create_dataset_jsons_aadish"
model_name = "llama"
model_adapter_output_home = "/home/kokil/ahmad/benchmarking_llms/projects/qlora_training/model_adapter_output_home"
classifier_training_pickle_output_home = "/home/kokil/ahmad/benchmarking_llms/projects/qlora_training/model_adapter_output_home/classifier_training_pickle_output_home"
INPUT_DIR = os.path.join(model_adapter_output_home, model_name, dataset_name)
OUTPUT_DIR = os.path.join(classifier_training_pickle_output_home, model_name, dataset_home)
# os.makedirs(OUTPUT_DIR, exist_ok=True)

lora_weights = INPUT_DIR

In [16]:
import os
data_path = os.path.join(dataset_home, f"{dataset_name}.json")
# data_path = "reddit.json"
from datasets import load_dataset
data = load_dataset("json", data_files=data_path)

In [17]:
train_val = data["train"].train_test_split(
    # test_size=200, shuffle=True, seed=42
    train_size = 900, test_size=100, shuffle=True, seed=42
)
train_data = (
    # train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    # train_val["test"].shuffle().map(generate_and_tokenize_prompt)
    train_val["test"].map(generate_and_tokenize_prompt)
)

# data = (
#     data["train"].shuffle().map(generate_and_tokenize_prompt)
# )

In [18]:
train_data[0].keys()

dict_keys(['input', 'instruction', 'output', 'input_ids', 'attention_mask'])

In [19]:
type(data)

datasets.dataset_dict.DatasetDict

In [20]:
train_data[0]['output']

'NEUTRAL'

In [23]:
model.config.use_cache = False
# train_stats = trainer.get_training_stats()
# print(train_stats)
print(datetime.now())
print("="*100)

2024-02-12 20:37:17.948205


In [24]:
train_data

Dataset({
    features: ['input', 'instruction', 'output', 'input_ids', 'attention_mask'],
    num_rows: 900
})

In [27]:
input_ids = train_data[0]["input_ids"]
input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)

In [28]:
from tqdm import tqdm

In [29]:
def get_last_hidden_state_of_last_token(model, input_ids):
    """
    The model can be either LlamaForCausalLM or PeftModelForCausalLM, so, we can't directly take model.model to get to the LlamaModel as in the latter case, model.model is actually LlamaForCausalLM, and we'll have to do model.model.model, so instead, we'll do model.get_decoder() as this function is only in and is not overridden when doing the addition with the adapter

    model = PeftModel.from_pretrained(
        model, # adapter_folder
        lora_weights,
        torch_dtype=torch.float16,
    )

    """

    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)

    # BaseModelOutputWithPast
    llama_base_model_output = model.get_decoder()(input_ids, output_hidden_states=True, return_dict=True)

    # the above is (1, token_length, 4096), where token_length is the no of input tokens, i.e. len(input_ids)

    # now if we want to get the last token's last hidden layer, we need to do
    return llama_base_model_output[0][-1]


from torch.utils.data import Dataset

# class CustomDataset(Dataset):
#     def __init__(self, data):
#         self.data = data

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         x, y = self.data[idx]
#         return {
#             'input': x,
#             'label': y
#         }


# class CustomDataset(Dataset):
#     def __init__(self, data):
#         self.last_hidden_state_of_last_token = data['last_hidden_state_of_last_token']
#         self.label = data['label']

#     def __len__(self):
#         return len(self.last_hidden_state_of_last_token)

#     def __getitem__(self, idx):
#         return {
#             'last_hidden_state_of_last_token': self.last_hidden_state_of_last_token[idx],
#             'label': self.label[idx]
#         }

class CustomDataset(Dataset):
    def __init__(self, data):
        self.last_hidden_state_of_last_token = data['last_hidden_state_of_last_token']
        self.label = data['label']

    def __len__(self):
        return len(self.last_hidden_state_of_last_token)

    def __getitem__(self, idx):
        return {
            'last_hidden_state_of_last_token': self.last_hidden_state_of_last_token[idx],
            'label': self.label[idx]
        }

    def __repr__(self):
        return f"Dataset(features={list(self[0].keys())}, num_rows={len(self)})"



# from datasets import Dataset, DatasetInfo, Value
# from datasets.builder import ArrowBasedBuilder

# class CustomDatasetBuilder(ArrowBasedBuilder):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#     def _info(self):
#         features = {
#             "last_hidden_state_of_last_token": Value('float32'),
#             "label": Value('int32')
#         }
#         return DatasetInfo(
#             features=features,
#         )

#     def _split_generators(self, dl_manager):
#         return [
#             self._split_generation(split_name='train', file_path=self.config.data_files['train']),
#             self._split_generation(split_name='validation', file_path=self.config.data_files['validation']),
#             self._split_generation(split_name='test', file_path=self.config.data_files['test'])
#         ]

#     def _split_generation(self, split_name, file_path):
#         return self._generate_examples(file_path)

#     def _generate_examples(self, file_path):
#         # Load your data from file or database
#         # For simplicity, let's assume data is a list of dictionaries
#         data = load_data(file_path)

#         # Yield individual examples
#         for idx, example in enumerate(data):
#             yield idx, {
#                 "last_hidden_state_of_last_token": example['last_hidden_state_of_last_token'],
#                 "label": example['label']
#             }


def convert_to_custom_dataset_for_classifier(model, dataset):
    # data_points = []
    data_points = {
        'last_hidden_state_of_last_token': [],
        'label': []
    }
    for i in tqdm(range(len(dataset))):
        # Assuming you extract x and y from each item
        last_hidden_state_of_last_token = get_last_hidden_state_of_last_token(model, dataset[i]["input_ids"])
        label = 0 if dataset[i]['output'] == 'NEUTRAL' else 1
        data_points['last_hidden_state_of_last_token'].append(last_hidden_state_of_last_token)
        data_points['label'].append(label)
    return CustomDataset(data_points)



In [30]:
base_or_finetuned = "finetuned"

In [31]:
train_dataset_classifier = convert_to_custom_dataset_for_classifier(model, train_data)

100%|██████████| 900/900 [02:30<00:00,  5.99it/s]


In [32]:
val_dataset_classifier = convert_to_custom_dataset_for_classifier(model, val_data)

100%|██████████| 100/100 [00:16<00:00,  6.10it/s]


In [33]:
from datasets import DatasetDict

# Combine train_dataset and val_dataset into a DatasetDict
dataset_dict_classifier = DatasetDict({"train": train_dataset_classifier, "val": val_dataset_classifier})


In [35]:
classifier_training_dataset_dict_path = os.path.join(OUTPUT_DIR, f"{model_name}={base_or_finetuned}={dataset_name}.dataset_dict")

In [36]:
classifier_training_dataset_dict_path

'/home/kokil/ahmad/benchmarking_llms/projects/qlora_training/create_dataset_jsons_aadish/llama=finetuned=gab.dataset_dict'

In [37]:
def save_custom_dataset_dict_for_classifier(dataset_dict_classifier, classifier_training_dataset_dict_path):
    torch.save(dataset_dict_classifier, classifier_training_dataset_dict_path)

In [38]:
def load_custom_dataset_dict_for_classifier(classifier_training_dataset_dict_path):
    dataset_dict_classifier = torch.load(classifier_training_dataset_dict_path)
    return dataset_dict_classifier

In [39]:
save_custom_dataset_dict_for_classifier(dataset_dict_classifier, classifier_training_dataset_dict_path)

In [40]:
www = load_custom_dataset_dict_for_classifier(classifier_training_dataset_dict_path)

In [41]:
www == dataset_dict_classifier

False

In [42]:
www

DatasetDict({
    train: Dataset(features=['last_hidden_state_of_last_token', 'label'], num_rows=900)
    val: Dataset(features=['last_hidden_state_of_last_token', 'label'], num_rows=100)
})

In [54]:
dataset_dict_classifier

DatasetDict({
    train: Dataset(features=['last_hidden_state_of_last_token', 'label'], num_rows=900)
    val: Dataset(features=['last_hidden_state_of_last_token', 'label'], num_rows=100)
})

In [63]:
for i in range(len(dataset_dict_classifier['train'])):
    print(i, (dataset_dict_classifier['train'][i]['last_hidden_state_of_last_token'] == www['train'][i]['last_hidden_state_of_last_token']).all().item())
    print(i, (dataset_dict_classifier['train'][i]['label'] == www['train'][i]['label']))

0 True
0 True
1 True
1 True
2 True
2 True
3 True
3 True
4 True
4 True
5 True
5 True
6 True
6 True
7 True
7 True
8 True
8 True
9 True
9 True
10 True
10 True
11 True
11 True
12 True
12 True
13 True
13 True
14 True
14 True
15 True
15 True
16 True
16 True
17 True
17 True
18 True
18 True
19 True
19 True
20 True
20 True
21 True
21 True
22 True
22 True
23 True
23 True
24 True
24 True
25 True
25 True
26 True
26 True
27 True
27 True
28 True
28 True
29 True
29 True
30 True
30 True
31 True
31 True
32 True
32 True
33 True
33 True
34 True
34 True
35 True
35 True
36 True
36 True
37 True
37 True
38 True
38 True
39 True
39 True
40 True
40 True
41 True
41 True
42 True
42 True
43 True
43 True
44 True
44 True
45 True
45 True
46 True
46 True
47 True
47 True
48 True
48 True
49 True
49 True
50 True
50 True
51 True
51 True
52 True
52 True
53 True
53 True
54 True
54 True
55 True
55 True
56 True
56 True
57 True
57 True
58 True
58 True
59 True
59 True
60 True
60 True
61 True
61 True
62 True
62 True
63 True
63 T

In [61]:
dataset_dict_classifier['train'][0]['label']

0

In [62]:
www['train'][0]['label']

0

In [52]:
dataset_dict_classifier['train'][0]['last_hidden_state_of_last_token'] == www['train'][0]['last_hidden_state_of_last_token']

tensor(True, device='cuda:11')

In [49]:
www['train'][0]['last_hidden_state_of_last_token']

tensor([[ 1.7729, -1.4340,  0.3075,  ..., -0.5573,  0.0064, -0.7626],
        [-2.1269, -1.0881,  0.1922,  ...,  0.5358, -1.3013, -0.1749],
        [-3.6840,  0.4955, -0.6878,  ...,  0.9461,  0.0840,  0.6209],
        ...,
        [ 0.0925, -0.5180,  2.2729,  ...,  0.3876, -0.2024, -2.4149],
        [-1.5827, -1.5720,  3.3173,  ...,  0.2413,  0.7279,  1.1369],
        [ 1.3438, -1.2516,  3.3698,  ..., -0.2226, -2.2946, -1.2497]],
       device='cuda:11')

In [ ]:
# def load_custom_dataset_for_classifier(dataset_classifier, ):
#     CustomDataset(torch.load(f'{split_name}_dataset.pt'))
#     # data_points =a {
#     #     'last_hidden_state_of_last_token': [],
#     #     'label': []
#     # }
#     # for data_point in dataset_classifier:
#     #     data_points['last_hidden_state_of_last_token'].append(data_point['last_hidden_state_of_last_token'])
#     #     data_points['label'].append(data_point['label'])

In [240]:
# def save_custom_dataset_for_classifier(dataset_classifier, ):
#     data_points = {
#         'last_hidden_state_of_last_token': [],
#         'label': []
#     }
#     for data_point in dataset_classifier:
#         data_points['last_hidden_state_of_last_token'].append(data_point['last_hidden_state_of_last_token'])
#         data_points['label'].append(data_point['label'])

In [ ]:
# loaded_combined_dataset = DatasetDict({
#     split_name: CustomDataset(torch.load(f'{split_name}_dataset.pt'))
#     for split_name in ['train', 'val']
# })


In [150]:
classifier_training_dataset_output_home = "classifier_training_dataset_output_home"

In [151]:
OUTPUT_DIR = os.path.join(classifier_training_dataset_output_home, model_name, base_or_finetuned)


In [152]:
OUTPUT_DIR

'classifier_training_dataset_output_home/llama/finetuned'

In [147]:
classifier_training_dataset_dict_path = os.path.join(OUTPUT_DIR, f"{model_name}={base_or_finetuned}={dataset_name}.dataset_classifier")

In [148]:
classifier_training_dataset_dict_path

'/home/kokil/ahmad/benchmarking_llms/projects/qlora_training/create_dataset_jsons_aadish/llama=finetuned=gab.dataset_classifier'

In [134]:
type(val_data)

datasets.arrow_dataset.Dataset

In [136]:
val_dataset_classifier = convert_to_custom_dataset_for_classifier(model, val_data)

100%|██████████| 100/100 [00:16<00:00,  6.11it/s]


In [59]:
len(model.get_decoder().layers)

32

In [54]:
input_ids.shape

torch.Size([1, 139])

In [29]:
# zzz = model(input_ids, output_hidden_states=True)
# zzz1 = model(input_ids, return_dict=True)
# zzz2 = model(input_ids, output_hidden_states=True, return_dict=True)
zzz3 = model.model(input_ids, output_hidden_states=True, return_dict=True)

In [34]:
model.lm_head

Linear(in_features=4096, out_features=32000, bias=False)

In [86]:
model.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p

In [38]:
zzz4 = model.get_decoder()(input_ids, output_hidden_states=True, return_dict=True)

In [80]:
input_ids.shape

torch.Size([1, 139])

In [41]:
input_ids.shape

torch.Size([1, 139])

In [40]:
zzz4.last_hidden_state.shape

torch.Size([1, 139, 4096])

In [84]:
zzz4.last_hidden_state[0].shape

torch.Size([139, 4096])

In [46]:
zzz4.last_hidden_state[0][-1].shape

torch.Size([4096])

In [60]:
zzz4.__dict__

{'last_hidden_state': tensor([[[ 1.7729, -1.4340,  0.3075,  ..., -0.5573,  0.0064, -0.7626],
          [-2.1269, -1.0881,  0.1922,  ...,  0.5358, -1.3013, -0.1749],
          [-3.6840,  0.4955, -0.6878,  ...,  0.9461,  0.0840,  0.6209],
          ...,
          [ 0.0925, -0.5180,  2.2729,  ...,  0.3876, -0.2024, -2.4149],
          [-1.5827, -1.5720,  3.3173,  ...,  0.2413,  0.7279,  1.1369],
          [ 1.3438, -1.2516,  3.3698,  ..., -0.2226, -2.2946, -1.2497]]],
        device='cuda:11'),
 'past_key_values': None,
 'hidden_states': (tensor([[[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
             5.9724e-05, -1.1957e-04],
           [ 2.5040e-02,  5.5618e-03, -6.8588e-03,  ...,  3.5934e-03,
            -2.4353e-02,  1.6769e-02],
           [-1.2650e-02, -1.8097e-02, -8.9264e-03,  ..., -1.7061e-03,
            -6.9504e-03,  1.1192e-02],
           ...,
           [-1.4565e-02, -2.0508e-02,  9.6035e-04,  ...,  3.5324e-03,
            -4.6921e-03, -2.1000e-03],
        

In [61]:
zzz4.__dict__.keys()

dict_keys(['last_hidden_state', 'past_key_values', 'hidden_states', 'attentions'])

In [66]:
inputs_embeds = model.get_decoder().embed_tokens(input_ids)

In [76]:
zzz4.last_hidden_state

tensor([[[ 1.7729, -1.4340,  0.3075,  ..., -0.5573,  0.0064, -0.7626],
         [-2.1269, -1.0881,  0.1922,  ...,  0.5358, -1.3013, -0.1749],
         [-3.6840,  0.4955, -0.6878,  ...,  0.9461,  0.0840,  0.6209],
         ...,
         [ 0.0925, -0.5180,  2.2729,  ...,  0.3876, -0.2024, -2.4149],
         [-1.5827, -1.5720,  3.3173,  ...,  0.2413,  0.7279,  1.1369],
         [ 1.3438, -1.2516,  3.3698,  ..., -0.2226, -2.2946, -1.2497]]],
       device='cuda:11')

In [77]:
zzz4.hidden_states[-1]

tensor([[[ 1.7729, -1.4340,  0.3075,  ..., -0.5573,  0.0064, -0.7626],
         [-2.1269, -1.0881,  0.1922,  ...,  0.5358, -1.3013, -0.1749],
         [-3.6840,  0.4955, -0.6878,  ...,  0.9461,  0.0840,  0.6209],
         ...,
         [ 0.0925, -0.5180,  2.2729,  ...,  0.3876, -0.2024, -2.4149],
         [-1.5827, -1.5720,  3.3173,  ...,  0.2413,  0.7279,  1.1369],
         [ 1.3438, -1.2516,  3.3698,  ..., -0.2226, -2.2946, -1.2497]]],
       device='cuda:11')

In [78]:
(zzz4.last_hidden_state == zzz4.hidden_states[-1]).all()

tensor(True, device='cuda:11')

##### last token's last hidden layer

In [93]:
zzz4.last_hidden_state[0][-1].shape

torch.Size([4096])

In [69]:
inputs_embeds.shape

torch.Size([1, 139, 4096])

In [73]:
(inputs_embeds == zzz4.hidden_states[0]).all()

tensor(True, device='cuda:0')

In [68]:
zzz4.hidden_states[0].shape

torch.Size([1, 139, 4096])

In [88]:
zzz4.last_hidden_state.shape

torch.Size([1, 139, 4096])

In [50]:
len(zzz4.hidden_states)

33

In [ ]:
def get_last_hidden_state()

In [31]:
zzz3.__dict__.keys()

dict_keys(['loss', 'logits', 'past_key_values', 'hidden_states', 'attentions'])

In [32]:
zzz3.last_hidden_state

AttributeError: 'CausalLMOutputWithPast' object has no attribute 'last_hidden_state'